In [ ]:
import requests
import json

# Function to split the list of IDs into chunks
def split_list(lst, n=100):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

# Function to make the Spotify API call
def fetch_artists_data(ids, token):
    """Fetch artists data from Spotify and return the response JSON."""
    headers = {
        'Authorization': f'Bearer {token}',
    }
    # Join the IDs with a comma for the API request
    ids_param = ','.join(ids)
    print(ids_param)
    response = requests.get(f'https://api.spotify.com/v1/artists?ids={ids_param}', headers=headers)
    return response.json()


# Load the Spotify artist IDs from a file
with open('C:\\Users\\Music\\Desktop\\PROJECTS\\Spotify Project\\96kArtistIDList.txt', 'r') as file:
    ids = file.read().splitlines()

token = 'BQAui-mmnKVGYXNb32vg-dlx9eH2Y9MzwbdgEEPN1X6C1L2hMRL1YFwgVh4yc1kx3HnQa16KV2l9GY7ESShFI3VxnsBbuAUZurvFIAYI5e5ImP7m3lcOWwmD2xIYHxEn63NIh0ykRn7YFkv0AoS6sFcv6wOoDD-ukkCJEOG--Bvq4cMAc1_tWCPbnOnCpwjyr994wG6W0LHuVGMA78buffkSEkia8f2Zzs7InXWVvPhWse7WQKzBkXYUoCZkL5FHoansJXsL-blUqtdZFgy5_nvJq-1TaNawX3oisBjD'

# Split the IDs into chunks of 100
id_chunks = list(split_list(ids, 50))

for i, chunk in enumerate(id_chunks):
    response_json = fetch_artists_data(chunk, token)
    
    # Check if the response contains an error
    if 'error' in response_json:
        print(f"Error fetching data for chunk {i+1}: {response_json['error']['message']}")
    else:
        # Save each chunk response into a separate JSON file
        with open(f'artists_data_chunk_{i+1}.json', 'w') as outfile:
            json.dump(response_json, outfile)

        print(f'Successfully fetched and saved data for chunk {i+1}')
